In [1]:
# Core Libraries
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Reinforcement Learning Libraries
import gym
from stable_baselines3 import DQN, A2C, PPO  # Example RL algorithms
import torch  # For deep learning
import gymnasium as gym  # Updated gym API for environments

# Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Constants
SHUFFLE_POINT = 0.25  # Reshuffle when 75% of the deck is used

In [27]:
# Create a single deck of cards
def create_deck():
    """
    Generates a standard deck of 52 cards.
    Each card is represented as a tuple: (value, suit).
    """
    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    values = ['10', '10', '10', '10', '10', '10', '10', '10', '10', 'J', 'Q', 'K', 'A']
    return [(value, suit) for suit in suits for value in values]

In [3]:
# Shuffle the deck
def shuffle_deck(deck):
    """
    Shuffles the given deck of cards.
    """
    random.shuffle(deck)
    return deck

In [4]:
# Deal a card with shuffle point logic
def deal_card(deck):
    """
    Deals the top card from the deck and tracks if the shuffle point (cut card) is reached.
    Does not reshuffle immediately if in the middle of a hand.
    """
    if len(deck) <= int(52 * SHUFFLE_POINT):
        print("Cut card reached! Reshuffle after this round.")
    return deck.pop()

In [5]:
# Deal initial hands
def deal_initial_hands(deck):
    """
    Deals two cards each to the player and the dealer.
    Reshuffles if there are not enough cards.
    """
    if len(deck) < 4:
        print("Not enough cards to deal initial hands. Reshuffling...")
        deck[:] = shuffle_deck(create_deck())
    player_hand = [deal_card(deck), deal_card(deck)]
    dealer_hand = [deal_card(deck), deal_card(deck)]
    return player_hand, dealer_hand

In [6]:
# Reshuffle after the round if the shuffle point is reached
def reshuffle_if_needed(deck):
    """
    Reshuffles the deck after the round if the shuffle point was reached.
    """
    if len(deck) <= int(52 * SHUFFLE_POINT):
        print("Reshuffling deck now...")
        deck[:] = shuffle_deck(create_deck())

In [7]:
def calculate_hand_value(hand):
    """
    Calculates the total value of a blackjack hand.
    Aces are treated dynamically as 1 or 11.
    """
    value = 0
    num_aces = 0

    # Map face cards to their values
    card_values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, 
                   '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10, 'A': 11}

    # Sum card values, counting Aces
    for card, suit in hand:
        value += card_values[card]
        if card == 'A':
            num_aces += 1

    # Adjust Aces from 11 to 1 as needed to avoid bust
    while value > 21 and num_aces > 0:
        value -= 10
        num_aces -= 1

    return value

In [8]:
def is_blackjack(hand):
    """
    Checks if a hand is a blackjack (Ace + 10-value card).
    """
    return len(hand) == 2 and calculate_hand_value(hand) == 21

In [9]:
# Function to check if splitting is possible
def can_split(hand):
    """
    Checks if the player's hand can be split (both cards have the same value).
    """
    if len(hand) == 2:
        card_values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
                       '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10, 'A': 11}
        return card_values[hand[0][0]] == card_values[hand[1][0]]
    return False

In [11]:
def determine_winner(player_value, dealer_value):
    """
    Compares the player's and dealer's hand values to determine the winner.
    """
    print("\n--- Final Results ---")
    if player_value > 21:
        print("You busted. Dealer wins.")
    elif dealer_value > 21:
        print("Dealer busted. You win!")
    elif player_value > dealer_value:
        print("You win!")
    elif player_value < dealer_value:
        print("Dealer wins.")
    else:
        print("It's a push!")

In [12]:
def initialize_bankroll(initial_amount=1000):
    """
    Initializes the player's bankroll with a default or specified amount.
    """
    return initial_amount

In [13]:
def place_wager(bankroll, wager_amount=10):
    """
    Deducts the wager from the bankroll and returns the updated bankroll and wager amount.
    """
    if wager_amount > bankroll:
        raise ValueError("Wager exceeds available bankroll.")
    return bankroll - wager_amount, wager_amount

In [14]:
def update_bankroll(bankroll, wager, outcome):
    """
    Updates the bankroll based on the game outcome.
    - Outcome: 'win', 'lose', or 'push'.
    """
    if outcome == 'win':
        return bankroll + (wager * 2)  # Player wins 2x their wager
    elif outcome == 'push':
        return bankroll + wager  # Player gets their wager back
    elif outcome == 'lose':
        return bankroll  # No refund on loss
    else:
        raise ValueError("Invalid outcome provided.")

In [15]:
def dealer_turn(dealer_hand, deck):
    """
    Handles the dealer's actions (hit or stand) based on blackjack rules.
    """
    while True:
        dealer_value = calculate_hand_value(dealer_hand)
        print(f"Dealer's Hand: {dealer_hand} | Value: {dealer_value}")

        # Dealer stands on hard 17 or above
        if dealer_value > 17 or (dealer_value == 17 and not any(card[0] == 'A' for card in dealer_hand)):
            print("Dealer stands.")
            break

        # Dealer hits on soft 17
        if dealer_value == 17 and any(card[0] == 'A' for card in dealer_hand):
            print("Dealer hits on soft 17.")
            dealer_hand.append(deal_card(deck))
            continue

        # Dealer hits below 17
        if dealer_value < 17:
            print("Dealer hits.")
            dealer_hand.append(deal_card(deck))

    return dealer_hand, dealer_value

In [29]:
def player_action(deck, initial_hand, bankroll, wager, max_splits=3):
    """
    Handles the player's actions (hit, stand, double down, or split), ensuring correct order for splits.
    Allows up to max_splits splits (resulting in up to max_splits + 1 hands).
    """
    player_hands = [initial_hand]  # Start with one hand
    wagers = [wager]  # Initial wager applied to all hands
    current_hand_index = 0  # Track which hand is being played
    splits_done = 0  # Keep track of the number of splits done

    while current_hand_index < len(player_hands):
        hand = player_hands[current_hand_index]
        print(f"\n--- Playing Hand {current_hand_index + 1} ---")

        # Loop to handle consecutive splits
        while True:
            # If the hand has only one card (due to a split), deal a second card
            if len(hand) == 1:
                hand.append(deal_card(deck))
                print(f"Hand {current_hand_index + 1} after dealing a new card: {hand} | Value: {calculate_hand_value(hand)}")
            else:
                print(f"Hand {current_hand_index + 1}: {hand} | Value: {calculate_hand_value(hand)}")

            # Check for splits if applicable
            if can_split(hand) and splits_done < max_splits:
                print(f"\nYou have a pair in Hand {current_hand_index + 1}! You can split.")
                while True:
                    split_choice = input(f"Do you want to split Hand {current_hand_index + 1}? (yes/no): ").strip().lower()
                    if split_choice in ['yes', 'no']:
                        break
                    print("Invalid input. Please enter 'yes' or 'no'.")
                if split_choice == 'yes':
                    if bankroll >= wagers[current_hand_index]:
                        print(f"Splitting Hand {current_hand_index + 1}... Deducting ${wagers[current_hand_index]} from your bankroll.")
                        bankroll -= wagers[current_hand_index]  # Deduct wager for the new hand

                        # Split the hand into two hands, each with one card
                        card1 = hand[0]
                        card2 = hand[1]
                        new_hand_1 = [card1]
                        new_hand_2 = [card2]
                        # Replace the current hand with the first split hand
                        player_hands[current_hand_index] = new_hand_1
                        # Add the new split hand to the end of the list
                        player_hands.append(new_hand_2)
                        wagers.append(wagers[current_hand_index])
                        splits_done += 1  # Increment splits done
                        print(f"Updated Bankroll after Split: ${bankroll}")
                        print(f"Current Hands: {player_hands}")

                        # Deal a new card to the current hand after splitting
                        hand = player_hands[current_hand_index]
                        hand.append(deal_card(deck))
                        print(f"Hand {current_hand_index + 1} after dealing a new card: {hand} | Value: {calculate_hand_value(hand)}")
                        # Continue the loop to check for further splits
                        continue
                    else:
                        print("Insufficient bankroll to split.")
                        break
                else:
                    # Player chose not to split
                    break
            else:
                # Can't split further
                break

        # Now, play the hand
        while True:
            hand_value = calculate_hand_value(hand)
            if hand_value > 21:
                print(f"Hand {current_hand_index + 1}: {hand} | Value: {hand_value}")
                print("Bust! Moving to the next hand.")
                break

            # Determine available actions
            if len(hand) == 2 and bankroll >= wagers[current_hand_index]:
                action_prompt = "Choose an action: 'hit', 'stand', or 'double down': "
            else:
                action_prompt = "Choose an action: 'hit' or 'stand': "

            action = input(action_prompt).strip().lower()

            if action == 'hit':
                print("You chose to hit.")
                hand.append(deal_card(deck))
                hand_value = calculate_hand_value(hand)
                print(f"Hand {current_hand_index + 1}: {hand} | Value: {hand_value}")
                if hand_value > 21:
                    print("Bust!")
                    break  # Exit decision loop after the action
            elif action == 'stand':
                print("You chose to stand.")
                break
            elif action == 'double down' and len(hand) == 2 and bankroll >= wagers[current_hand_index]:
                print(f"You chose to double down. Doubling wager to ${wagers[current_hand_index] * 2}.")
                bankroll -= wagers[current_hand_index]  # Deduct the additional wager
                wagers[current_hand_index] *= 2  # Double the wager
                hand.append(deal_card(deck))  # One final card
                hand_value = calculate_hand_value(hand)
                print(f"Hand {current_hand_index + 1} after double down: {hand} | Value: {hand_value}")
                if hand_value > 21:
                    print("Bust!")
                break  # Automatically stand after doubling down
            else:
                print("Invalid action or insufficient bankroll for double down.")

        # Move to the next hand
        current_hand_index += 1

    return player_hands, wagers, bankroll



In [ ]:
# Main execution
if __name__ == "__main__":
    # Initialize bankroll
    bankroll = initialize_bankroll()

    while bankroll > 0:  # Continue as long as player has money
        print(f"\nCurrent Bankroll: ${bankroll}")

        # Place wager
        try:
            bankroll, wager = place_wager(bankroll)
            print(f"Wager placed: ${wager}")
        except ValueError as e:
            print(e)
            break

        # Initialize and shuffle the deck
        deck = shuffle_deck(create_deck())

        # Deal initial hands
        player_hand, dealer_hand = deal_initial_hands(deck)

        # Display initial hands
        print("\n--- Initial Hands ---")
        print("Player's Hand:", player_hand, "| Value:", calculate_hand_value(player_hand))
        print("Dealer's Face-Up Card:", dealer_hand[0])  # Show only one dealer card

        # Check for insurance opportunity
        insurance_wager = 0
        if dealer_hand[0][0] == 'A':  # Dealer is showing an Ace
            print("Dealer is showing an Ace. Insurance is available.")
            while True:
                insurance_choice = input("Do you want to take insurance? (yes/no): ").strip().lower()
                if insurance_choice in ['yes', 'no']:
                    break
                print("Invalid input. Please enter 'yes' or 'no'.")
            if insurance_choice == 'yes':
                insurance_wager = wager / 2
                bankroll -= insurance_wager
                print(f"Insurance wager placed: ${insurance_wager}")

        # Check if dealer has a blackjack
        if is_blackjack(dealer_hand):
            print("\nDealer has blackjack!")
            print(f"Dealer's Hand: {dealer_hand}")

            if insurance_wager:
                # Dealer has blackjack; insurance pays 2:1
                insurance_payout = insurance_wager * 2  # 2:1 payout
                bankroll += insurance_payout + insurance_wager  # Add winnings + return insurance wager
                print(f"Insurance pays 2:1. You win ${insurance_payout} from insurance.")

            if is_blackjack(player_hand):
                print("It's a push! Both you and the dealer have blackjack.")
                outcome = 'push'
            else:
                print("Dealer wins with a blackjack!")
                outcome = 'lose'

            # Deduct main wager based on outcome
            bankroll = update_bankroll(bankroll, wager, outcome)
            print(f"Updated Bankroll: ${bankroll}")
            while True:
                continue_game = input("Do you want to play another round? (yes/no): ").strip().lower()
                if continue_game in ['yes', 'no']:
                    break
                print("Invalid input. Please enter 'yes' or 'no'.")
            if continue_game != 'yes':
                break
            continue

        # Check if player has a blackjack
        if is_blackjack(player_hand):
            print("\nYou have blackjack!")
            winnings = int(wager * 1.5)  # Blackjack pays 3:2
            bankroll += winnings + wager  # Add winnings + original wager
            print(f"Blackjack pays 3:2. You win ${winnings}!")
            print(f"Dealer's Full Hand: {dealer_hand} | Value: {calculate_hand_value(dealer_hand)}")
            print(f"Updated Bankroll: ${bankroll}")
            while True:
                continue_game = input("Do you want to play another round? (yes/no): ").strip().lower()
                if continue_game in ['yes', 'no']:
                    break
                print("Invalid input. Please enter 'yes' or 'no'.")
            if continue_game != 'yes':
                break
            continue

        # Player's turn (with split handling capped at 4 hands)
        player_hands, wagers, bankroll = player_action(deck, player_hand, bankroll, wager, max_splits=3)

        # Dealer's turn for each hand
        for i, hand in enumerate(player_hands):
            if calculate_hand_value(hand) <= 21:
                print(f"\n--- Dealer's Turn for Hand {i + 1} ---")
                dealer_hand, dealer_value = dealer_turn(dealer_hand, deck)

                # Determine winner
                if dealer_value > 21 or calculate_hand_value(hand) > dealer_value:
                    outcome = 'win'
                elif calculate_hand_value(hand) < dealer_value:
                    outcome = 'lose'
                else:
                    outcome = 'push'

                determine_winner(calculate_hand_value(hand), dealer_value)

                # Update bankroll per hand
                bankroll = update_bankroll(bankroll, wagers[i], outcome)
                print(f"Hand {i + 1} Outcome: {outcome.capitalize()} | Updated Bankroll: ${bankroll}")

        # If all hands are busted, reveal dealer's hand
        if all(calculate_hand_value(hand) > 21 for hand in player_hands):
            print("\n--- Dealer's Final Hand ---")
            print(f"Dealer's Hand: {dealer_hand} | Value: {calculate_hand_value(dealer_hand)}")

        # Ask if player wants to continue
        while True:
            continue_game = input("Do you want to play another round? (yes/no): ").strip().lower()
            if continue_game in ['yes', 'no']:
                break
            print("Invalid input. Please enter 'yes' or 'no'.")

        if continue_game != 'yes':
            break

    print("\nGame Over. Final Bankroll:", bankroll)



Current Bankroll: $1000
Wager placed: $10

--- Initial Hands ---
Player's Hand: [('10', 'Hearts'), ('A', 'Hearts')] | Value: 21
Dealer's Face-Up Card: ('10', 'Hearts')

You have blackjack!
Blackjack pays 3:2. You win $15!
Dealer's Full Hand: [('10', 'Hearts'), ('Q', 'Clubs')] | Value: 20
Updated Bankroll: $1015


Do you want to play another round? (yes/no):  yes



Current Bankroll: $1015
Wager placed: $10

--- Initial Hands ---
Player's Hand: [('10', 'Clubs'), ('10', 'Hearts')] | Value: 20
Dealer's Face-Up Card: ('10', 'Hearts')

--- Playing Hand 1 ---
Hand 1: [('10', 'Clubs'), ('10', 'Hearts')] | Value: 20

You have a pair in Hand 1! You can split.


Do you want to split Hand 1? (yes/no):  yes


Splitting Hand 1... Deducting $10 from your bankroll.
Updated Bankroll after Split: $995
Current Hands: [[('10', 'Clubs')], [('10', 'Hearts')]]
Hand 1 after dealing a new card: [('10', 'Clubs'), ('Q', 'Diamonds')] | Value: 20
Hand 1: [('10', 'Clubs'), ('Q', 'Diamonds')] | Value: 20

You have a pair in Hand 1! You can split.


...

You chose to stand.

--- Playing Hand 3 ---
Hand 3 after dealing a new card: [('Q', 'Diamonds'), ('10', 'Spades')] | Value: 20


Choose an action: 'hit', 'stand', or 'double down':  stand


You chose to stand.

--- Playing Hand 4 ---
Hand 4 after dealing a new card: [('10', 'Spades'), ('10', 'Hearts')] | Value: 20


Choose an action: 'hit', 'stand', or 'double down':  stand


You chose to stand.

--- Dealer's Turn for Hand 1 ---
Dealer's Hand: [('10', 'Hearts'), ('10', 'Diamonds')] | Value: 20
Dealer stands.

--- Final Results ---
You win!
Hand 1 Outcome: Win | Updated Bankroll: $995

--- Dealer's Turn for Hand 2 ---
Dealer's Hand: [('10', 'Hearts'), ('10', 'Diamonds')] | Value: 20
Dealer stands.

--- Final Results ---
It's a push!
Hand 2 Outcome: Push | Updated Bankroll: $1005

--- Dealer's Turn for Hand 3 ---
Dealer's Hand: [('10', 'Hearts'), ('10', 'Diamonds')] | Value: 20
Dealer stands.

--- Final Results ---
It's a push!
Hand 3 Outcome: Push | Updated Bankroll: $1015

--- Dealer's Turn for Hand 4 ---
Dealer's Hand: [('10', 'Hearts'), ('10', 'Diamonds')] | Value: 20
Dealer stands.

--- Final Results ---
It's a push!
Hand 4 Outcome: Push | Updated Bankroll: $1025
